<h1><center>WUDC Analytics</center></h1>

### 1. Importing Relevant Modules and Reading File

In [ ]:
"""
Original information from:
https://wudckorea.calicotab.com/2021/tab/team
https://wudc2022.calicotab.com/wudc/tab/team
https://wudc2023.calicotab.com/wudc/tab/team
"""

import pandas as pd
import numpy as np

from itertools import groupby

#For summary display - not a necessary component
from IPython.display import display, HTML

df = pd.read_excel(r"{WUDC File}")

df

### 2. Normalising Points Table
A specific team with a specific round relate to a single value - their result. <br> 
Done as the three columns: Team | Round | Points

In [ ]:
#Saving the points per team for each of the 9 rounds, each as their own dataframe
def simplify_function(roundnumber_string):  
    df_info = df[["team",roundnumber_string]]
    df_info.insert(0, 'round', roundnumber_string)
    df_info = df_info.rename({roundnumber_string: 'points'}, axis=1)
    df_info = df_info[['team', 'round', 'points']]
    return df_info
     
df_r1 = simplify_function("R1")
df_r2 = simplify_function("R2")
df_r3 = simplify_function("R3")
df_r4 = simplify_function("R4")
df_r5 = simplify_function("R5")
df_r6 = simplify_function("R6")
df_r7 = simplify_function("R7")
df_r8 = simplify_function("R8")
df_r9 = simplify_function("R9")

#Concatenate (union) the created tables and set a fresh index
score_table1 = pd.concat([df_r1, df_r2, df_r3, df_r4, df_r5, df_r6, df_r7, df_r8, df_r9]).reset_index()
score_table1 = score_table1.drop('index', axis=1)

#Turn positions into their scores for numeric analysis
score_table2 = score_table1.replace({'points': {'1st': 3, '2nd': 2, '3rd': 1, '4th': 0}}) #Replace string to numbers
score_table2['points'] = score_table2['points'].apply(pd.to_numeric, errors='coerce') #Change to numeric type

score_table2

#Try a specific team to test it works:
#score_table2[score_table2["team"]=="LSE A"]

### 3. Testing 'The Radicals'
Criteria for 'Radicals' are:
1. Debated all 9 rounds
2. Wins >= Losses
3. Ordered by total number of wins (1st) and losses (4th)
4. Secondarily order by evenness of wins and losses - measured by wins minus losses

In [ ]:
#Crosstab on points by team, margins created the 'All' column"
cross_score_table = pd.crosstab(score_table2['team'], score_table2['points'],margins=True)

#Adding the analytic columns
cross_score_table.insert(4, 'wins_and_losses', cross_score_table.loc[:,[0.0,3.0]].sum(axis=1))
cross_score_table.insert(5, 'win>=loss', np.where(cross_score_table[0.0]<=cross_score_table[3.0], 'True', 'False'))
cross_score_table.insert(6, 'evenness', cross_score_table[3.0] - cross_score_table[0.0])

#Filtering data
cross_score_filter1a = cross_score_table[cross_score_table['All']== 9] #Debated in all 9 rounds
cross_score_filter2a = cross_score_filter1a[cross_score_filter1a['win>=loss']=='True'] #Won more than lost

#Print 1:
#cross_score_filter2a.sort_values(['wins_and_losses','evenness'], ascending=[False, True]).head(5)

### 4. Testing 'The Centrists'
Criteria for 'Centrists' are:
1. Debated all 9 rounds
2. Ordered by total number of 2nds and 3rds - note will just use lowest number of wins and losses
3. Evenness of 2nds and 3rds as secondary order

In [ ]:
#Add new evenness measure to the existing unfiltered table
cross_score_table.insert(7, 'centrist_evenness', cross_score_table[2.0] - cross_score_table[1.0])
cross_score_table['centrist_evenness'] = cross_score_table['centrist_evenness'].abs()

In [ ]:
#Filter
cross_score_filter1b = cross_score_table[cross_score_table['All']== 9]

#Print 2:
#cross_score_filter1b.sort_values(['wins_and_losses','centrist_evenness'], ascending=[True, True]).head(5)

### 5. Testing 'Silver Tongued Streak'
Criteria for 'Silver Tongued Streak' are:
1. As many wins in a row
2. Secondary order of as many wins and 2nds in a row

In [ ]:
#For tests on functions:
#single_value_data = "1st1st1st4th1st2nd1st1st1st"


def convert_score_string_to_list(results_string):
    a = ",".join(results_string[i : i + 3] for i in range(0, len(results_string), 3))
    results_lst = a.split(",")
    return results_lst

def most_firsts_in_row(results_string):
    team_results_lst = convert_score_string_to_list(results_string)
    
    d = dict() #Creation of dictionary
    
    for k, v in groupby(team_results_lst):
        d.setdefault(k, []).append(len(list(v))) #Populating the dictionary using the list based on 

    a = {k:v for k, v in d.items() if k in ('1st')} #Selecting only the 1sts in a row
    
    try: 
        return max(a["1st"]) #Finding the max 1st in a row
    except:
        return 0

def seconds_to_firsts(results_string):
    team_results_lst = convert_score_string_to_list(results_string)
    #Converts 2nds to 1sts
    converted_lst = list(map(lambda x: x.replace('2nd', '1st'), team_results_lst))
    back_to_string = ''.join(converted_lst)
    return back_to_string

In [ ]:
round_lst = ["R1","R2","R3","R4","R5","R6","R7","R8","R9"]

#Moves all rounds information, for a given team, into their 2 new columns
df.insert(13, 'wins_in_row', df.loc[:,round_lst].sum(axis=1))
df.insert(14, 'wins_and_2nds_in_row', df.loc[:,round_lst].sum(axis=1))

#Apply the functions to test the wins in row
df['wins_in_row'] = df['wins_in_row'].apply(most_firsts_in_row)
df['wins_and_2nds_in_row'] = df['wins_and_2nds_in_row'].apply(seconds_to_firsts).apply(most_firsts_in_row)

#Print 3:
#df.sort_values(['wins_in_row','wins_and_2nds_in_row'], ascending=[False, False]).head(5)

### 6. Summary

In [ ]:
print('\033[1m' + '\033[4m' + "Top 5 Radicals:")
display(cross_score_filter2a.sort_values(['wins_and_losses','evenness'], ascending=[False, True]).head(5))

print('\033[1m' + '\033[4m' + "Top 5 Centrists:")
display(cross_score_filter1b.sort_values(['wins_and_losses','centrist_evenness'], ascending=[True, True]).head(5))

print('\033[1m' + '\033[4m' + "Top 5 Silver Tongued Streak:")
display(df.sort_values(['wins_in_row','wins_and_2nds_in_row'], ascending=[False, False]).head(5))